In [205]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from IPython.display import Markdown, display

from dotenv import load_dotenv

load_dotenv('/workspaces/ml-learning/src/airtable-chatbot/.env', override=True)

import nest_asyncio
nest_asyncio.apply()

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


https://llamahub.ai/l/airtable?from=all

Base ID: The path in the URL that begins with app  appfjm76R87oVG3BP  
Table ID: The path in the URL that begins with tbl tblkBhmgAFBRGmNRL  

https://support.airtable.com/docs/creating-personal-access-tokens  




In [ ]:
!pip install pyairtable

## Custom Airtable reader

In [157]:
"""Airtable reader."""
from typing import List

from llama_index.core.readers.base import BaseReader
from llama_index.core.schema import Document, BaseNode, TextNode
from pyairtable import Table, Api, Base
import pandas as pd
import json
import uuid

BUILD_CLUB_MEMBERS_AIRTABLE_COLUMNS = {
    "skills": "What are your areas of expertise you have (select max 4 please)",
    "name": "Name",
    "linkedin_url": "What's the link to your LinkedIn?",
    "build_project": "What will you build"
}


class CustomAirtableReader(BaseReader):
    """Airtable reader. Reads data from a table in a base.

    Args:
        api_key (str): Airtable API key.
    """

    def __init__(self, api_key: str, base_id: str, table_id: str) -> None:
        """Initialize Airtable reader."""

        self.api = Api(api_key)
        self.base_id = base_id
        self.table_id = table_id 
        self._data = None
        self._fields = None

    @property
    def data(self, reload=False):
        if (self._data is None or reload==True): 
            self._data = self._load_data()
        return self._data
    
    @property
    def fields(self, reload=False):
        reloaded = False
        if (self._data is None or reload==True): 
            self._data = self._load_data()
            reloaded=True
        if (self._fields is None or reloaded==True):
            all_records = self._data
            self._fields = [record.get("fields", {}) for record in all_records]
        return self._fields

    def _load_data(self) -> List[dict]:
        table = self.api.table(self.base_id, self.table_id)
        self._data = table.all()
        return self._data

    # def load_data(self, base_id: str, table_id: str) -> List[Document]:
    #     """Load data from a table in a base

    #     Args:
    #         table_id (str): Table ID.
    #         base_id (str): Base ID.
    #     Returns:
    #         List[Document]: List of documents.
    #     """

    #     table = self.api.table(base_id, table_id)

    #     # table = Table(self.api_key, base_id, table_id)
    #     all_records = table.all()

    #     # Extract the 'fields' content from each element
    #     fields = [item['fields'] for item in all_records]

    #     documents = []
    #     for field in fields:
    #         # Copy the fields dictionary to extra_info
    #         extra_info = field.copy()

    #         # Keys to be removed from text and metadata (we just don't care about it)
    #         keys_to_remove = ['Profile picture']
    #         for key in keys_to_remove:
    #             extra_info.pop(key, None)  # The None argument ensures no error if the key doesn't exist

    #         text_dict = extra_info.copy()

    #         keys_to_remove = ['What will you build']

    #         # Remove the keys from extra_info if they exist
    #         for key in keys_to_remove:
    #             extra_info.pop(key, None)  # The None argument ensures no error if the key doesn't exist

    #         # Now extra_info contains the fields data without the specified keys
    #         print(extra_info)

    #         text_for_node = json.dumps(text_dict, indent=2)
    #         print(text_for_node)

    #         document = Document(text=text_for_node, extra_info=extra_info)
    #         documents.append(document)

    #     return documents
    
    def extract_documents(self) -> List[Document]:

        # if (self.data is None): 
        #     _ = self._load_data()

        all_records = self.data

        # Extract the 'fields' content from each element
        fields = [item['fields'] for item in all_records]

        documents = []
        for field in fields:
            # Copy the fields dictionary to extra_info
            extra_info = field.copy()

            # Keys to be removed from text and metadata (we just don't care about it)
            keys_to_remove = ['Profile picture']
            for key in keys_to_remove:
                extra_info.pop(key, None)  # The None argument ensures no error if the key doesn't exist

            text_dict = extra_info.copy()

            keys_to_remove = ['What will you build']

            # Remove the keys from extra_info if they exist
            for key in keys_to_remove:
                extra_info.pop(key, None)  # The None argument ensures no error if the key doesn't exist

            # Now extra_info contains the fields data without the specified keys
            print(extra_info)

            text_for_node = json.dumps(text_dict, indent=2)
            print(text_for_node)

            document = Document(text=text_for_node, extra_info=extra_info)
            documents.append(document)

        return documents

    def extract_nodes(self) -> List[BaseNode]:
        documents = self.extract_documents()

        nodes = [ TextNode(text=d.text, metadata=d.metadata) for d in documents]
        return nodes
    
    def extract_skills(self) -> set:
        all_skills = set()
        for record in self.data:
            fields = record.get("fields", {})
            print(fields.get("Name", "Unknown name"), fields.get(BUILD_CLUB_MEMBERS_AIRTABLE_COLUMNS["skills"], "No skills"))
            skills = fields.get(BUILD_CLUB_MEMBERS_AIRTABLE_COLUMNS["skills"], [])
            all_skills.update(skills)
        return all_skills
    
    def extract_rows(self) -> List[dict]:

        transformed = []
        all_records = self.data
        for record in all_records:
            fields = record.get("fields", {})
            skills = fields.get("What are your areas of expertise you have (select max 4 please)", [])
            transformed_detail = {
                "id": record.get("id", str(uuid.uuid4())),
                "member_name": fields.get("Name", ""),
                "linkedin_url": fields.get("What's the link to your LinkedIn?", ""),
                "skill_1": skills[0] if len(skills)>0 else "",
                "skill_2": skills[1] if len(skills)>1 else "",
                "skill_3": skills[2] if len(skills)>2 else "",
                "skill_4": skills[3] if len(skills)>3 else "",
                "build_project": fields.get("What will you build", ""),
            }
            print(json.dumps(transformed_detail, indent=2))
            transformed.append(transformed_detail)
        return transformed
        

    
    def get_airtable_df(self, base_id: str, table_id: str)-> pd.DataFrame:

        # if (self.data is None): 
        #     _ = self._load_data()

        all_records = self.data

        # Extract the 'fields' content from each element
        fields = [item['fields'] for item in all_records]

        # Create a DataFrame from the extracted 'fields'
        df = pd.DataFrame(fields)

        return df

In [158]:
from llama_index.core.readers.download import download_loader
import os

# AirtableReader = download_loader('AirtableReader')

AIRTABLE_TOKEN=os.environ['AIRTABLE_TOKEN']
AIRTABLE_BASE_ID=os.environ['AIRTABLE_BASE_ID']
AIRTABLE_TABLE_ID=os.environ['AIRTABLE_TABLE_ID']

reader = CustomAirtableReader(AIRTABLE_TOKEN, table_id=AIRTABLE_TABLE_ID,base_id=AIRTABLE_BASE_ID)


In [160]:
skills = reader.extract_skills()

Mike Gardiner ['Backend software dev', 'Front end software dev']
Marina Ritchie ['Go to market', 'Idea validating']
Cameron Bogatez ['Product management', 'Designer']
Atena Pegler ['AI / ML specialist researcher', 'Designer', 'Product management', 'Idea validating', 'Go to market']
Sumit Saggar ['Front end software dev', 'Product management', 'Domain expert']
luka gamulin ['Backend software dev', 'Front end software dev', 'AI / ML specialist researcher']
Imad Hashmi ['Backend software dev', 'Go to market', 'Domain expert', 'AI Engineer']
Bastien Cabirou ['Front end software dev', 'Backend software dev', 'AI / ML specialist researcher']
Lynn thompson  ['Go to market', 'Domain expert']
Siva Kalyan ['AI / ML specialist researcher', 'Designer']
Brent Maxwell ['Product management', 'Go to market']
Paul van der Linden ['Product management', 'AI Engineer', 'Go to market']
mark ghiasy ['Go to market']
Mani Batra ['AI Engineer', 'Backend software dev', 'Product management']
Abhinit Kumar Ambast

In [122]:
skills

{'AI / ML specialist researcher',
 'AI Engineer',
 'Backend software dev',
 'Designer',
 'Domain expert',
 'Front end software dev',
 'Go to market',
 'Idea validating',
 'Product management'}

In [123]:
blob  = reader._load_data()

In [124]:
blob

[{'id': 'rec0F550OZwfSFwIL',
  'createdTime': '2024-02-11T05:28:43.000Z',
  'fields': {'Name': 'Mike Gardiner',
   'Profile picture': [{'id': 'attgeBMA8bKI4rNg4',
     'width': 4624,
     'height': 3468,
     'url': 'https://v5.airtableusercontent.com/v3/u/25/25/1707976800000/_dWAajttcKBAWpW-jT0Ecg/E3IJ7GqrdKynI4VRoi4PGU13zFJXnW1R3i7l_AGgTrlz0sPun8KvyqbMQu7FXN6Le2KUISM96z1tiBJw6GhgPs6JBROb4hswh9jMmnMwDjBu0aqMt25pS3dbXubH1UussWKwllggb0hGKPdX4N-tmw/gTVjho7zd9ZlnbiyC1RzzYpFSlGpjHPBIGRQ4CtAGQg',
     'filename': '20230501_142648.jpg',
     'size': 6286835,
     'type': 'image/jpeg',
     'thumbnails': {'small': {'url': 'https://v5.airtableusercontent.com/v3/u/25/25/1707976800000/JSBf7M8pOzIznaUDVnnwdA/rM88yqPP1E2WiYTgIohpOEAFlwFQcgHFydTTYhMPLz51T16mlko_YqS_DhdNFA1lIYoAEP9kq3jxNm53Znw6WwiKNgyKvCCg3uL_gi37FB2z4fDymaD1rWOYxZyPO2NgYyd7o1nhRSnbDerSmcsBbA/BOPKBqwypzAGnIXtnWLeNoPv5Vh80iWz7-wAdGk-3FU',
       'width': 27,
       'height': 36},
      'large': {'url': 'https://v5.airtableusercontent

In [133]:
documents = reader.extract_documents()

{'Name': 'Mike Gardiner', "What's the link to your LinkedIn?": 'https://www.linkedin.com/in/mrmikeg', 'What are your areas of expertise you have (select max 4 please)': ['Backend software dev', 'Front end software dev']}
{
  "Name": "Mike Gardiner",
  "What's the link to your LinkedIn?": "https://www.linkedin.com/in/mrmikeg",
  "What are your areas of expertise you have (select max 4 please)": [
    "Backend software dev",
    "Front end software dev"
  ],
  "What will you build": "\ud83d\udc1d AI coach for game development teams to enable them to build, play and learn together."
}
{'Name': 'Marina Ritchie', "What's the link to your LinkedIn?": 'https://www.linkedin.com/in/marinaritchie/', 'What are your areas of expertise you have (select max 4 please)': ['Go to market', 'Idea validating']}
{
  "Name": "Marina Ritchie",
  "What's the link to your LinkedIn?": "https://www.linkedin.com/in/marinaritchie/",
  "What are your areas of expertise you have (select max 4 please)": [
    "Go to 

In [68]:
df = reader.get_airtable_df(table_id=AIRTABLE_TABLE_ID,base_id=AIRTABLE_BASE_ID)

In [70]:
df['Profile picture'][0]

[{'id': 'attgeBMA8bKI4rNg4',
  'width': 4624,
  'height': 3468,
  'url': 'https://v5.airtableusercontent.com/v3/u/25/25/1707969600000/MeOjuzWewKwleVlz3H1k0w/KUj2dICWdHQ5DE5qWQaWaRIjEndJ-UNSPIEDxFONUyec4a4J9wkUBZ5V4lWDZ-LQJkfcjnaM6Q0Tgut1uyt3T7jR2F4CvD4_kNRppXq9hpALvp1cIMTRFrSDRyldqFgIkqr83DMMCF93Hi53quzFBw/27oNSIX_pwlBFFC6Cnr9wBdwsOZ03UwdIBcxwTiIZtk',
  'filename': '20230501_142648.jpg',
  'size': 6286835,
  'type': 'image/jpeg',
  'thumbnails': {'small': {'url': 'https://v5.airtableusercontent.com/v3/u/25/25/1707969600000/BVO6AnRpsiDRxKEpeJG2qg/20sajUoX52o1q0jS256SJG2Z_gNIZNTzaGONjjcQdnXiGQFNC20qlZLASktq1NTIkBkbQRQP0pFnTNNB6zBbA9261MOaVFrmfTB5ekdtwACFQFU04g-jLrt-HEyMSYl1isMdKV0iSPUWuNA7ePWanw/WT_n5Qbt8DsO9fkq28kZHJiqtxGErH-4vcDjzSoFwN8',
    'width': 27,
    'height': 36},
   'large': {'url': 'https://v5.airtableusercontent.com/v3/u/25/25/1707969600000/Z0tNrD8dmVj7V8tr5FtRzw/jLdLkYC1EkohfOST1RgDzRtt2Sf1Ryte407V0x58oKYuyRy2OQ6fIKnXHePNp9Xkrq9HYORcLA3EcXttqlRDP8y2ojv3zP_lr0UzYVLxR5nYVBD

## Vector store index

In [135]:
from llama_index.core.indices import VectorStoreIndex
# from llama_index.llms import OpenAI

nodes = reader.extract_nodes()



{'Name': 'Mike Gardiner', "What's the link to your LinkedIn?": 'https://www.linkedin.com/in/mrmikeg', 'What are your areas of expertise you have (select max 4 please)': ['Backend software dev', 'Front end software dev']}
{
  "Name": "Mike Gardiner",
  "What's the link to your LinkedIn?": "https://www.linkedin.com/in/mrmikeg",
  "What are your areas of expertise you have (select max 4 please)": [
    "Backend software dev",
    "Front end software dev"
  ],
  "What will you build": "\ud83d\udc1d AI coach for game development teams to enable them to build, play and learn together."
}
{'Name': 'Marina Ritchie', "What's the link to your LinkedIn?": 'https://www.linkedin.com/in/marinaritchie/', 'What are your areas of expertise you have (select max 4 please)': ['Go to market', 'Idea validating']}
{
  "Name": "Marina Ritchie",
  "What's the link to your LinkedIn?": "https://www.linkedin.com/in/marinaritchie/",
  "What are your areas of expertise you have (select max 4 please)": [
    "Go to 

In [136]:
index = VectorStoreIndex(nodes=nodes)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "

### semantic query engine from modules

In [213]:
# semantic query engine from indexer
from modules.reader import CustomAirtableReader
from modules.indexer import Indexer
import os

# AirtableReader = download_loader('AirtableReader')

AIRTABLE_TOKEN=os.environ['AIRTABLE_TOKEN']
AIRTABLE_BASE_ID=os.environ['AIRTABLE_BASE_ID']
AIRTABLE_TABLE_ID=os.environ['AIRTABLE_TABLE_ID']

reader = CustomAirtableReader(AIRTABLE_TOKEN, table_id=AIRTABLE_TABLE_ID,base_id=AIRTABLE_BASE_ID)

indexer = Indexer(reader, "MembersIndex")

In [197]:
response = indexer.semantic_query_engine.query("Who works on ML algorithms?")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [198]:
pprint(response)

Response(response='John Moriarty works on ML algorithms, as indicated by his '
                  'expertise as an "AI / ML specialist researcher".',
         source_nodes=[NodeWithScore(node=TextNode(id_='b4febc4f-cf9c-4cb7-8d43-bd966efecd0c', embedding=None, metadata={'Name': 'Albert Lie', "What's the link to your LinkedIn?": 'https://www.linkedin.com/in/alberttriadrian/', 'What are your areas of expertise you have (select max 4 please)': ['Backend software dev', 'AI Engineer', 'Idea validating']}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='{\n  "Name": "Albert Lie",\n  "What\'s the link to your LinkedIn?": "https://www.linkedin.com/in/alberttriadrian/",\n  "What are your areas of expertise you have (select max 4 please)": [\n    "Backend software dev",\n    "AI Engineer",\n    "Idea validating"\n  ],\n  "What will you build": "AI employees for freight forwarders"\n}', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\

In [206]:
db_response = indexer.db_query_engine.query("Who works on ML algorithms?")

INFO:llama_index.core.indices.struct_store.sql_retriever:> Table desc str: Table 'build_club_members' has columns: id (VARCHAR(256)), member_name (VARCHAR(256)), linkedin_url (VARCHAR(512)), skill_1 (VARCHAR(128)), skill_2 (VARCHAR(128)), skill_3 (VARCHAR(128)), skill_4 (VARCHAR(128)), build_project (VARCHAR(4096)), and foreign keys: .
> Table desc str: Table 'build_club_members' has columns: id (VARCHAR(256)), member_name (VARCHAR(256)), linkedin_url (VARCHAR(512)), skill_1 (VARCHAR(128)), skill_2 (VARCHAR(128)), skill_3 (VARCHAR(128)), skill_4 (VARCHAR(128)), build_project (VARCHAR(4096)), and foreign keys: .
> Table desc str: Table 'build_club_members' has columns: id (VARCHAR(256)), member_name (VARCHAR(256)), linkedin_url (VARCHAR(512)), skill_1 (VARCHAR(128)), skill_2 (VARCHAR(128)), skill_3 (VARCHAR(128)), skill_4 (VARCHAR(128)), build_project (VARCHAR(4096)), and foreign keys: .
> Table desc str: Table 'build_club_members' has columns: id (VARCHAR(256)), member_name (VARCHAR(25

In [207]:
from pprint import pprint
pprint(db_response)

Response(response="I'm sorry, but there was an error in retrieving the "
                  'information. Please try again later.',
         source_nodes=[NodeWithScore(node=TextNode(id_='cfa1fb46-ad7a-4402-8eb5-d3fd74bd7023', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='Error: Statement \'"SELECT member_name, linkedin_url FROM build_club_members WHERE skill_1 = \\\'AI / ML specialist researcher\\\' \\n                OR skill_2 = \\\'AI / ML specialist researcher\\\' \\n                OR skill_3 = \\\'AI / ML specialist researcher\\\' \\n                OR skill_4 = \\\'AI / ML specialist researcher\\\'"\' is invalid SQL.', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=None)],
         metadata={'cfa1fb46-ad7a-4402-8eb5-d3fd74bd7023': {},
                   'sql_query': '"SELECT member_name, linkedin_url FROM

In [ ]:
sql_query = '"SELECT member_name, linkedin_url FROM '
                                "build_club_members WHERE skill_1 = 'AI / ML "
                                "specialist researcher' \n"
                                "                OR skill_2 = 'AI / ML "
                                "specialist researcher' \n"
                                "                OR skill_3 = 'AI / ML "
                                "specialist researcher' \n"
                                "                OR skill_4 = 'AI / ML "
                                'specialist researcher\'"'

In [210]:
from sqlalchemy.sql import text
with indexer.engine.connect() as con:


    statement = text("""SELECT member_name, linkedin_url FROM build_club_members WHERE skill_1 = 'AI / ML specialist researcher' \n
                                     OR skill_2 = 'AI / ML specialist researcher' \n                OR skill_3 = 'AI / ML specialist researcher' \n
                                     OR skill_4 = 'AI / ML """)

    results = con.execute(statement)

    for row in results:
        print(row)

('Atena Pegler', 'https://www.linkedin.com/in/atenakouchaki')
('luka gamulin', 'https://www.linkedin.com/in/luka-jos%C3%A9-gamulin-63a332b2/')
('Bastien Cabirou', 'https://www.linkedin.com/in/bastiencabirou/')
('Siva Kalyan', 'https://www.linkedin.com/in/siva-kalyan-3a34a455/')
('Abhinit Kumar Ambastha', 'https://www.linkedin.com/in/abhinitambastha/')
('Arman Abrahamyan', 'https://www.linkedin.com/in/armanabrahamyan/')
('Joyce Yu', 'https://www.linkedin.com/in/jyu1/')
('Ada Gao ', 'https://www.linkedin.com/in/ada-gao-44813576')
('Dan Butcher', 'https://www.linkedin.com/in/danbutcher1981/')
('Matthew K Ma', 'www.linkedin.com/in/mattma1970')
('Shadab Hussain', 'https://www.linkedin.com/in/techwithshadab/')
('Viren Bachani', 'https://www.linkedin.com/in/virenbachani/')
('Roel Van de Paar', 'https://www.linkedin.com/in/roelvandepaar/')
('Nguyen Ngoc Doan PHAN', 'https://www.linkedin.com/in/jasmine-phan-8676b8194/')
('John Hu', 'https://www.linkedin.com/in/john-hu1/')
('Kyle Yang', 'https:/

## semantic query engine

In [137]:

from llama_index.core import ServiceContext
from llama_index.llms.openai import OpenAI
from llama_index.core.settings import Settings


llm = OpenAI(model="gpt-4", temperature=0)

semantic_query_engine = index.as_query_engine(llm=llm)


In [138]:

response = semantic_query_engine.query('What is Chamira\'s project?')
response.response

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


'Chamira\'s project is an "AI News Bias Detector & AI SMS Marketing Customer Acquisition platform".'

In [139]:
from pprint import pprint
pprint(response.source_nodes[0].node.text)

('{\n'
 '  "Name": " Chamira Gamage",\n'
 '  "What\'s the link to your LinkedIn?": '
 '"https://www.linkedin.com/in/chamirag?utm_source=share&utm_campaign=share_via&utm_content=profile&utm_medium=ios_app",\n'
 '  "What are your areas of expertise you have (select max 4 please)": [\n'
 '    "Idea validating",\n'
 '    "Go to market",\n'
 '    "Product management"\n'
 '  ],\n'
 '  "What will you build": "AI News Bias Detector & AI SMS Marketing Customer '
 'Acquisition platform"\n'
 '}')


In [236]:
response = semantic_query_engine.query('Which members are software engineers? please provide their name')

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [237]:
from pprint import pprint
pprint(response.response)

('The members who are software engineers are Prabhjot Singh Sodhi and Steve '
 'James.')


In [238]:
db_response = db_query_engine.query('Which members are software engineers? please provide their name')

INFO:llama_index.core.indices.struct_store.sql_retriever:> Table desc str: Table 'build_club_members' has columns: id (VARCHAR(256)), member_name (VARCHAR(256)), linkedin_url (VARCHAR(512)), skill_1 (VARCHAR(128)), skill_2 (VARCHAR(128)), skill_3 (VARCHAR(128)), skill_4 (VARCHAR(128)), build_project (VARCHAR(4096)), and foreign keys: .
> Table desc str: Table 'build_club_members' has columns: id (VARCHAR(256)), member_name (VARCHAR(256)), linkedin_url (VARCHAR(512)), skill_1 (VARCHAR(128)), skill_2 (VARCHAR(128)), skill_3 (VARCHAR(128)), skill_4 (VARCHAR(128)), build_project (VARCHAR(4096)), and foreign keys: .
> Table desc str: Table 'build_club_members' has columns: id (VARCHAR(256)), member_name (VARCHAR(256)), linkedin_url (VARCHAR(512)), skill_1 (VARCHAR(128)), skill_2 (VARCHAR(128)), skill_3 (VARCHAR(128)), skill_4 (VARCHAR(128)), build_project (VARCHAR(4096)), and foreign keys: .
> Table desc str: Table 'build_club_members' has columns: id (VARCHAR(256)), member_name (VARCHAR(25

In [239]:
pprint(db_response)

Response(response='The members who are software engineers are: Mike Gardiner, '
                  'Sumit Saggar, Luka Gamulin, Imad Hashmi, Bastien Cabirou, '
                  'Mani Batra, Abhinit Kumar Ambastha, Steve James, Arman '
                  'Abrahamyan, Joe Cleaves, James Keogh, Frederik Bull-Larsen, '
                  'Joyce Yu, Eduard Serra Tuneu, Raphael Bismut, Aishwarya '
                  'Sahu, Peter Hayes, Itay Guetta, Edgar Haond, Vivian Shen, '
                  'Laurence Qi, Thai Hoa Do, Jesus Flores, Nathan, Jamie '
                  'Tuppack, Jad Goss, Ricky Sydney, Anna Zhang, Jerry Zhao, '
                  'Thiago Bomfim, Cameron Smith, Duc Tuan Nguyen, Jasper '
                  'Goalpure, Jack Wakem, Julian Bright, Amos Tan, Richard '
                  'Mulyawan Wibowo, Joshua Theeuf, Nick Buckingham-Jones, '
                  'Laurence Hendry, Jason Yu, Andrei Ziganshin, Mikhail '
                  'Eremeev, Juan David Bernal Puerto, Aaron Xie, Kun Hong,

In [142]:
pprint(response.source_nodes[0].node.text)
pprint(response.source_nodes[1].node.text)
pprint(len(response.source_nodes))

('{\n'
 '  "Name": "Prabhjot Singh Sodhi",\n'
 '  "What\'s the link to your LinkedIn?": '
 '"https://www.linkedin.com/in/prabhjotsodhi/",\n'
 '  "What are your areas of expertise you have (select max 4 please)": [\n'
 '    "Front end software dev",\n'
 '    "Backend software dev",\n'
 '    "AI Engineer"\n'
 '  ],\n'
 '  "What will you build": "Working with Atech Design Solutions"\n'
 '}')
('{\n'
 '  "Name": "Steve James",\n'
 '  "What\'s the link to your LinkedIn?": "n/a",\n'
 '  "What are your areas of expertise you have (select max 4 please)": [\n'
 '    "Backend software dev",\n'
 '    "Front end software dev",\n'
 '    "AI Engineer",\n'
 '    "Product management"\n'
 '  ],\n'
 '  "What will you build": "Numerous AI experiments ranging from RAG to Auto '
 'coding"\n'
 '}')
2


In [143]:
response = semantic_query_engine.query('Tell all the member names that are working on projects for children')

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [144]:
from pprint import pprint
pprint(response.response)

('There are no members working on projects for children according to the '
 'provided information.')


In [21]:
pprint(response.source_nodes[0].node.text)

'{}'


In [21]:

pprint(response.source_nodes[0].node.text)

('": \'https://www.linkedin.com/in/shaemckenna/\', \'What are your areas of '
 "expertise you have (select max 4 please)': ['AI Engineer', 'Product "
 "management', 'AI / ML specialist researcher'], 'What will you build': "
 '"I am planning on launching a startup in the edtech space. I always thought '
 'classrooms were a really ineffective way of teaching, one teacher for 25+ '
 "students just doesn't seem effective no matter how good a teacher may be. "
 'ChatGPT revealed the effectiveness of Open-Domain Question Answering '
 'however, it is still too unstructured to make a true dent in the educational '
 'space. This is where I see project Minerva coming into play. I aim to '
 'leverage LLM technology through an education-first architecture, to '
 'revolutionize how students learn around the world.')


## selector for tools

https://docs.llamaindex.ai/en/latest/examples/query_transformations/query_transform_cookbook.html#  


In [22]:
from llama_index.core.tools import ToolMetadata

tool_choices = [
    ToolMetadata(
        name="member_skills",
        description=("This tool retrieves "),
    ),
    ToolMetadata(
        name="covid_wiki",
        description=("This tool contains the Wikipedia page about COVID-19"),
    ),
    ToolMetadata(
        name="covid_tesla",
        description=("This tool contains the Wikipedia page about apples"),
    ),
]

https://docs.llamaindex.ai/en/latest/understanding/putting_it_all_together/structured_data.html

## sql queries

In [162]:
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    select,
    column,
)

engine = create_engine("sqlite:///:memory:")
default_schema = MetaData()


In [147]:
# # create city SQL table
# table_name = "city_stats"
# city_stats_table = Table(
#     table_name,
#     default_schema,
#     Column("city_name", String(16), primary_key=True),
#     Column("population", Integer),
#     Column("country", String(16), nullable=False),
# )
# default_schema.create_all(engine)

In [163]:
from sqlalchemy import (
    MetaData,
    Engine
)

def design_build_club_members_table(engine: Engine, schema: MetaData) -> Table:
    metadata_obj = MetaData()
    table_name = "build_club_members"
    build_club_members = Table(
        table_name,
        metadata_obj,
        Column("id", String(256), primary_key=True),
        Column("member_name", String(256)),
        Column("linkedin_url", String(512)),
        Column("skill_1", String(128)),
        Column("skill_2", String(128)),
        Column("skill_3", String(128)),
        Column("skill_4", String(128)),
        Column("build_project", String(4096))
    )
    metadata_obj.create_all(engine)
    return build_club_members

build_club_members_table = design_build_club_members_table(engine, default_schema)

default_schema.create_all(engine)


### transform into rows

In [154]:
# def transform_build_club_member_details(fields: List[dict]):
#     transformed = []
#     for field in fields:
#         skills = field.get("What are your areas of expertise you have (select max 4 please)", [])
#         transformed_detail = {
#             "member_name": field.get("Name", ""),
#             "linkedin_url": field.get("What's the link to your LinkedIn?", ""),
#             "skill_1": skills[0] if len(skills)>0 else "",
#             "skill_2": skills[1] if len(skills)>1 else "",
#             "skill_3": skills[2] if len(skills)>2 else "",
#             "skill_4": skills[3] if len(skills)>3 else "",
#             "build_project": field.get("What will you build", ""),
#         }
#         print(json.dumps(transformed_detail, indent=2))
#         transformed.append(transformed_detail)
#     return transformed

# rows = transform_build_club_member_details(reader.fields)
# rows

{
  "member_name": "Mike Gardiner",
  "linkedin_url": "https://www.linkedin.com/in/mrmikeg",
  "skill_1": "Backend software dev",
  "skill_2": "Front end software dev",
  "skill_3": "",
  "skill_4": "",
  "build_project": "\ud83d\udc1d AI coach for game development teams to enable them to build, play and learn together."
}
{
  "member_name": "Marina Ritchie",
  "linkedin_url": "https://www.linkedin.com/in/marinaritchie/",
  "skill_1": "Go to market",
  "skill_2": "Idea validating",
  "skill_3": "",
  "skill_4": "",
  "build_project": "I provide advise to business owners on AI Strategy and train team on ChatGPT. I would like to build an App for Galleries listings in SA and nationally using power of AI."
}
{
  "member_name": "Cameron Bogatez",
  "linkedin_url": "https://www.linkedin.com/in/cameron-bogatez123/",
  "skill_1": "Product management",
  "skill_2": "Designer",
  "skill_3": "",
  "skill_4": "",
  "build_project": "AI Steve, an AI product consultant that generates ideas to make y

[{'member_name': 'Mike Gardiner',
  'linkedin_url': 'https://www.linkedin.com/in/mrmikeg',
  'skill_1': 'Backend software dev',
  'skill_2': 'Front end software dev',
  'skill_3': '',
  'skill_4': '',
  'build_project': '🐝 AI coach for game development teams to enable them to build, play and learn together.'},
 {'member_name': 'Marina Ritchie',
  'linkedin_url': 'https://www.linkedin.com/in/marinaritchie/',
  'skill_1': 'Go to market',
  'skill_2': 'Idea validating',
  'skill_3': '',
  'skill_4': '',
  'build_project': 'I provide advise to business owners on AI Strategy and train team on ChatGPT. I would like to build an App for Galleries listings in SA and nationally using power of AI.'},
 {'member_name': 'Cameron Bogatez',
  'linkedin_url': 'https://www.linkedin.com/in/cameron-bogatez123/',
  'skill_1': 'Product management',
  'skill_2': 'Designer',
  'skill_3': '',
  'skill_4': '',
  'build_project': "AI Steve, an AI product consultant that generates ideas to make your product easie

In [165]:
rows = reader.extract_rows()
rows

{
  "id": "rec0F550OZwfSFwIL",
  "member_name": "Mike Gardiner",
  "linkedin_url": "https://www.linkedin.com/in/mrmikeg",
  "skill_1": "Backend software dev",
  "skill_2": "Front end software dev",
  "skill_3": "",
  "skill_4": "",
  "build_project": "\ud83d\udc1d AI coach for game development teams to enable them to build, play and learn together."
}
{
  "id": "rec0O9YHldw1qeWNI",
  "member_name": "Marina Ritchie",
  "linkedin_url": "https://www.linkedin.com/in/marinaritchie/",
  "skill_1": "Go to market",
  "skill_2": "Idea validating",
  "skill_3": "",
  "skill_4": "",
  "build_project": "I provide advise to business owners on AI Strategy and train team on ChatGPT. I would like to build an App for Galleries listings in SA and nationally using power of AI."
}
{
  "id": "rec0RM3X4rpOWXWmS",
  "member_name": "Cameron Bogatez",
  "linkedin_url": "https://www.linkedin.com/in/cameron-bogatez123/",
  "skill_1": "Product management",
  "skill_2": "Designer",
  "skill_3": "",
  "skill_4": ""

[{'id': 'rec0F550OZwfSFwIL',
  'member_name': 'Mike Gardiner',
  'linkedin_url': 'https://www.linkedin.com/in/mrmikeg',
  'skill_1': 'Backend software dev',
  'skill_2': 'Front end software dev',
  'skill_3': '',
  'skill_4': '',
  'build_project': '🐝 AI coach for game development teams to enable them to build, play and learn together.'},
 {'id': 'rec0O9YHldw1qeWNI',
  'member_name': 'Marina Ritchie',
  'linkedin_url': 'https://www.linkedin.com/in/marinaritchie/',
  'skill_1': 'Go to market',
  'skill_2': 'Idea validating',
  'skill_3': '',
  'skill_4': '',
  'build_project': 'I provide advise to business owners on AI Strategy and train team on ChatGPT. I would like to build an App for Galleries listings in SA and nationally using power of AI.'},
 {'id': 'rec0RM3X4rpOWXWmS',
  'member_name': 'Cameron Bogatez',
  'linkedin_url': 'https://www.linkedin.com/in/cameron-bogatez123/',
  'skill_1': 'Product management',
  'skill_2': 'Designer',
  'skill_3': '',
  'skill_4': '',
  'build_projec

### insert rows into table

In [166]:
from sqlalchemy import insert

rows = reader.extract_rows()

for row in rows:
    stmt = insert(build_club_members_table).values(**row)
    with engine.begin() as connection:
        cursor = connection.execute(stmt)

{
  "id": "rec0F550OZwfSFwIL",
  "member_name": "Mike Gardiner",
  "linkedin_url": "https://www.linkedin.com/in/mrmikeg",
  "skill_1": "Backend software dev",
  "skill_2": "Front end software dev",
  "skill_3": "",
  "skill_4": "",
  "build_project": "\ud83d\udc1d AI coach for game development teams to enable them to build, play and learn together."
}
{
  "id": "rec0O9YHldw1qeWNI",
  "member_name": "Marina Ritchie",
  "linkedin_url": "https://www.linkedin.com/in/marinaritchie/",
  "skill_1": "Go to market",
  "skill_2": "Idea validating",
  "skill_3": "",
  "skill_4": "",
  "build_project": "I provide advise to business owners on AI Strategy and train team on ChatGPT. I would like to build an App for Galleries listings in SA and nationally using power of AI."
}
{
  "id": "rec0RM3X4rpOWXWmS",
  "member_name": "Cameron Bogatez",
  "linkedin_url": "https://www.linkedin.com/in/cameron-bogatez123/",
  "skill_1": "Product management",
  "skill_2": "Designer",
  "skill_3": "",
  "skill_4": ""

### setup query engine

In [168]:
from llama_index.core import SQLDatabase

sql_database = SQLDatabase(engine, include_tables=["build_club_members"])

In [184]:
from llama_index.core.prompts.base import PromptTemplate
from llama_index.core.prompts.prompt_type import PromptType

TEXT_TO_SQL_TMPL = (
    """Given an input question, first create a syntactically correct {dialect} 
    query to run, then look at the results of the query and return the answer. 
    You can order the results by a relevant column to return the most 
    interesting examples in the database.

    Pay attention to use only the column names that you can see in the schema
    description.
    Be careful to not query for columns that do not exist.
    Pay attention to which column is in which table.
    Also, qualify column names with the table name when needed.
    You are required to use the following format, each taking one line:

    Question: Question here
    SQLQuery: SQL Query to run
    SQLResult: Result of the SQLQuery
    Answer: Final answer here

    Only use tables listed below.
    {schema}

    In table "build_club_members", each member can list 4 skills in columns skill_1, skill_2, skill_3, skill_4.
    Here's the set of possible values these columns can take:
    'AI / ML specialist researcher',
    'AI Engineer',
    'Backend software dev',
    'Designer',
    'Domain expert',
    'Front end software dev',
    'Go to market',
    'Idea validating',
    'Product management'
    
    So when the Question mentions a semantically similar skill, please translate into one of the existing skill or a combination of similar skills.
    For example: 
    Question: "What builders are software engineers?"
    SQLQuery: "SELECT member_name, linkedin_url FROM build_club_members WHERE skill_1 = 'Backend software dev' OR skill_1 = 'Front end software dev' 
                OR skill_2 = 'Backend software dev' OR skill_2 = 'Front end software dev'
                OR skill_3 = 'Backend software dev' OR skill_3 = 'Front end software dev'
                OR skill_4 = 'Backend software dev' OR skill_4 = 'Front end software dev'

    Question: "What builders are into UX Design?"
    SQLQuery: "SELECT member_name, linkedin_url FROM build_club_members WHERE skill_1 = 'Designer'
                OR skill_2 = 'Designer' 
                OR skill_3 = 'Designer'
                OR skill_4 = 'Designer'

    Question: {query_str}
    SQLQuery: 
    """
)

TEXT_TO_SQL_PROMPT = PromptTemplate(
    TEXT_TO_SQL_TMPL,
    prompt_type=PromptType.TEXT_TO_SQL,
)

In [ ]:
text_to_sql_prompt_build_club_members = """You excel at retrieving information from sql table using a mix of SQL knowledge and natural language processing skills. 
You will get a natural language query, and will skillfully translate that

For example:
Natural
"""


In [185]:
from llama_index.core.query_engine import NLSQLTableQueryEngine

db_query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=["build_club_members"],
    text_to_sql_prompt=TEXT_TO_SQL_PROMPT,
    verbose=True
)
query_str = "Which builders are software engineer?"
response = db_query_engine.query(query_str)

INFO:llama_index.core.indices.struct_store.sql_retriever:> Table desc str: Table 'build_club_members' has columns: id (VARCHAR(256)), member_name (VARCHAR(256)), linkedin_url (VARCHAR(512)), skill_1 (VARCHAR(128)), skill_2 (VARCHAR(128)), skill_3 (VARCHAR(128)), skill_4 (VARCHAR(128)), build_project (VARCHAR(4096)), and foreign keys: .
> Table desc str: Table 'build_club_members' has columns: id (VARCHAR(256)), member_name (VARCHAR(256)), linkedin_url (VARCHAR(512)), skill_1 (VARCHAR(128)), skill_2 (VARCHAR(128)), skill_3 (VARCHAR(128)), skill_4 (VARCHAR(128)), build_project (VARCHAR(4096)), and foreign keys: .
> Table desc str: Table 'build_club_members' has columns: id (VARCHAR(256)), member_name (VARCHAR(256)), linkedin_url (VARCHAR(512)), skill_1 (VARCHAR(128)), skill_2 (VARCHAR(128)), skill_3 (VARCHAR(128)), skill_4 (VARCHAR(128)), build_project (VARCHAR(4096)), and foreign keys: .
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Requ

In [186]:
response.response

'Some of the builders who are software engineers include Michael Holborn, Tew Tawan, Guillermo Gette, Louka Ewington-Pitsos, Ivan Pua, Yan Paing Oo, Gabriel Poussif, Rohan Agarwal, Francisco Galarza, Peter Ebeid, Santiago Bravo, Gayan Kincy Kulatilleke, Ashfaqul Bari (Ash), Javed Sheikh, Aliaksei Kuncevic, Pete Field, George Wang, Akhil Ramani, Ben Skinner, Prabhjot Singh Sodhi, Michal Wanto, Rebecca Williams, Gabriel Wong, Ben Hosken, Loke Li Voon, Phuong Anh Dinh, Anton Alice Vice, Dr Sam Donegan, Xavier Andueza, Ben Jackson, Ryan Cushen, Elliott Lovell, Sheldon Lin, Stephen Young, Kaushal Ottem, Nish, Ryan Taylor, Libin Joseph, Leopold Lucas, Katrin Schmid, Kevork Simon Mouradian, Oren Bahari, Trevor Brunton, Garry Singh, Miles Johnson, Tomas Dowd, Laurie Nicol, Samanvay Karambhe, Aninda Saha, Shern Ren Tee, Yujin Wu, The An, Pranav Dhoolia, Mikhail Eremeev, Simon Jones.'

In [187]:
pprint(response)

Response(response='Some of the builders who are software engineers include '
                  'Michael Holborn, Tew Tawan, Guillermo Gette, Louka '
                  'Ewington-Pitsos, Ivan Pua, Yan Paing Oo, Gabriel Poussif, '
                  'Rohan Agarwal, Francisco Galarza, Peter Ebeid, Santiago '
                  'Bravo, Gayan Kincy Kulatilleke, Ashfaqul Bari (Ash), Javed '
                  'Sheikh, Aliaksei Kuncevic, Pete Field, George Wang, Akhil '
                  'Ramani, Ben Skinner, Prabhjot Singh Sodhi, Michal Wanto, '
                  'Rebecca Williams, Gabriel Wong, Ben Hosken, Loke Li Voon, '
                  'Phuong Anh Dinh, Anton Alice Vice, Dr Sam Donegan, Xavier '
                  'Andueza, Ben Jackson, Ryan Cushen, Elliott Lovell, Sheldon '
                  'Lin, Stephen Young, Kaushal Ottem, Nish, Ryan Taylor, Libin '
                  'Joseph, Leopold Lucas, Katrin Schmid, Kevork Simon '
                  'Mouradian, Oren Bahari, Trevor Brunton, Garry Si

In [62]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

db_query_engine_tool = QueryEngineTool(
    query_engine=db_query_engine,
    metadata=ToolMetadata(
        name="db_query_engine",
        description="useful for when you want to answer queries about city population",
    ),
)

In [63]:
semantic_query_engine_tool = QueryEngineTool(
    query_engine=semantic_query_engine,
    metadata=ToolMetadata(
        name="semantic_query_engine",
        description="useful for when you want to answer queries about members projects and biography",
    ),
)

In [64]:
tools = [db_query_engine_tool, semantic_query_engine_tool]

## agent

In [227]:
# semantic query engine from indexer
from modules.reader import CustomAirtableReader
from modules.indexer import Indexer
import os

# AirtableReader = download_loader('AirtableReader')

AIRTABLE_TOKEN=os.environ['AIRTABLE_TOKEN']
AIRTABLE_BASE_ID=os.environ['AIRTABLE_BASE_ID']
AIRTABLE_TABLE_ID=os.environ['AIRTABLE_TABLE_ID']

reader = CustomAirtableReader(AIRTABLE_TOKEN, table_id=AIRTABLE_TABLE_ID,base_id=AIRTABLE_BASE_ID)

indexer = Indexer(reader, "MembersIndex")

In [228]:
indexer.tools

{
  "id": "rec0F550OZwfSFwIL",
  "member_name": "Mike Gardiner",
  "linkedin_url": "https://www.linkedin.com/in/mrmikeg",
  "skill_1": "Backend software dev",
  "skill_2": "Front end software dev",
  "skill_3": "",
  "skill_4": "",
  "build_project": "\ud83d\udc1d AI coach for game development teams to enable them to build, play and learn together."
}
{
  "id": "rec0O9YHldw1qeWNI",
  "member_name": "Marina Ritchie",
  "linkedin_url": "https://www.linkedin.com/in/marinaritchie/",
  "skill_1": "Go to market",
  "skill_2": "Idea validating",
  "skill_3": "",
  "skill_4": "",
  "build_project": "I provide advise to business owners on AI Strategy and train team on ChatGPT. I would like to build an App for Galleries listings in SA and nationally using power of AI."
}
{
  "id": "rec0RM3X4rpOWXWmS",
  "member_name": "Cameron Bogatez",
  "linkedin_url": "https://www.linkedin.com/in/cameron-bogatez123/",
  "skill_1": "Product management",
  "skill_2": "Designer",
  "skill_3": "",
  "skill_4": ""

In [231]:
from llama_index.agent.openai import OpenAIAgent
llm = OpenAI(model='gpt-4', temperature=0)
agent = OpenAIAgent.from_tools(indexer.tools, verbose=True)

In [221]:
from llama_index.agent.openai import OpenAIAssistantAgent
assistant = OpenAIAssistantAgent.from_new(name='Henri', instructions='You are a helpful assistant', tools=indexer.tools, verbose=True)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/assistants "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/assistants "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/assistants "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/assistants "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/threads "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/threads "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/threads "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/threads "HTTP/1.1 200 OK"


In [240]:
response = agent.chat('Which members are software engineers? please provide their name')

Added user message to memory: Which members are software engineers? please provide their name
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
=== Calling Function ===
Calling function: db_query_engine with args: {
  "input": "software engineers"
}
INFO:llama_index.core.indices.struct_store.sql_retriever:> Table desc str: Table 'build_club_members' has columns: id (VARCHAR(256)), member_name (VARCHAR(256)), linkedin_url (VARCHAR(512)), skill_1 (VARCHAR(128)), skill_2 (VARCHAR(128)), skill_3 (VARCHAR(128)), skill_4 (VARCHAR(128)), build_project (VARCHAR(4096)), and foreign keys: .
> Table desc str: Table 'build_club_members' has columns: id (VARCHAR(256)), member_name (VARCHAR(256)), linkedin_url (VARCHAR(512)

In [233]:
_ = response

In [234]:
pprint(response)

Response(response='John Moriarty works on ML algorithms, as indicated by his '
                  'expertise as an "AI / ML specialist researcher".',
         source_nodes=[NodeWithScore(node=TextNode(id_='b4febc4f-cf9c-4cb7-8d43-bd966efecd0c', embedding=None, metadata={'Name': 'Albert Lie', "What's the link to your LinkedIn?": 'https://www.linkedin.com/in/alberttriadrian/', 'What are your areas of expertise you have (select max 4 please)': ['Backend software dev', 'AI Engineer', 'Idea validating']}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='{\n  "Name": "Albert Lie",\n  "What\'s the link to your LinkedIn?": "https://www.linkedin.com/in/alberttriadrian/",\n  "What are your areas of expertise you have (select max 4 please)": [\n    "Backend software dev",\n    "AI Engineer",\n    "Idea validating"\n  ],\n  "What will you build": "AI employees for freight forwarders"\n}', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\

In [223]:
assistant.chat('How many members have software engineering as a skill?')

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/threads/thread_BYQ2USJ02WTL6SRM01v1BMKD/messages "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/threads/thread_BYQ2USJ02WTL6SRM01v1BMKD/messages "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/threads/thread_BYQ2USJ02WTL6SRM01v1BMKD/messages "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/threads/thread_BYQ2USJ02WTL6SRM01v1BMKD/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/threads/thread_BYQ2USJ02WTL6SRM01v1BMKD/runs "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/threads/thread_BYQ2USJ02WTL6SRM01v1BMKD/runs "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/threads/thread_BYQ2USJ02WTL6SRM01v1BMKD/runs "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/threads/thread_BYQ2USJ02WTL6SRM01v1BMKD/runs "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://api.openai.com/v1/threads/thread_BYQ2USJ02WTL6SRM01v1BMKD/run

AgentChatResponse(response='I apologize for the continued difficulties. It appears that I am unable to retrieve the specific information about members with the skill "software engineering" at this time. If there\'s anything else you\'d like to know or if you have another question, please feel free to ask.', sources=[ToolOutput(content="I'm sorry, but there was an error in executing the SQL statement. The statement is invalid. Please check the syntax and try again.", tool_name='db_query_engine', raw_input={'input': 'software engineering'}, raw_output=Response(response="I'm sorry, but there was an error in executing the SQL statement. The statement is invalid. Please check the syntax and try again.", source_nodes=[NodeWithScore(node=TextNode(id_='4ff919f9-3d30-4e9d-b163-233c80594618', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='Error: Statement \'"SELECT member_name, linkedin_url FROM build_club_members WHERE skill_

In [224]:
_ = response

In [67]:
assistant.chat('What is Chamira\'s project?')

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/threads/thread_Umpu3JAEgxR4RFNpOKvjA6jo/messages "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/threads/thread_Umpu3JAEgxR4RFNpOKvjA6jo/messages "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/threads/thread_Umpu3JAEgxR4RFNpOKvjA6jo/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/threads/thread_Umpu3JAEgxR4RFNpOKvjA6jo/runs "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/threads/thread_Umpu3JAEgxR4RFNpOKvjA6jo/runs "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/threads/thread_Umpu3JAEgxR4RFNpOKvjA6jo/runs "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://api.openai.com/v1/threads/thread_Umpu3JAEgxR4RFNpOKvjA6jo/runs/run_cwFWu891UVXNWs7gnhWGmfhZ "HTTP/1.1 200 OK"
HTTP Request: GET https://api.openai.com/v1/threads/thread_Umpu3JAEgxR4RFNpOKvjA6jo/runs/run_cwFWu891UVXNWs7gnhWGmfhZ "HTTP/1.1 200 OK"
HTTP Request: GET https://api.opena

AgentChatResponse(response='Chamira is working on two projects: "Product Rank Pro" and "Survey Smarter".', sources=[ToolOutput(content='Chamira is working on "Product Rank Pro" and "Survey Smarter".', tool_name='semantic_query_engine', raw_input={'input': "What is Chamira's project?"}, raw_output=Response(response='Chamira is working on "Product Rank Pro" and "Survey Smarter".', source_nodes=[NodeWithScore(node=TextNode(id_='af8e7183-d2dc-4507-8d1a-d1f4c85557a5', embedding=None, metadata={'Name': ' Chamira Gamage', 'Profile picture': [{'id': 'attGlTYut6dbxa5OG', 'width': 400, 'height': 400, 'url': 'https://v5.airtableusercontent.com/v3/u/25/25/1707962400000/w1gSMCcklXLXTz4cBOGkLg/-kOwerrcucP6xSsQmAW2krjI7VHQ9dLeQN2K-Rz9lUgjeWxMEvJqraPP1S5iB9jjgW3qRo7Z3Job92iMu6FE2NVr6hHrAJvCY7IehcY8BaNf9csdTf24TFlhXygqKMVlFo-4qY8WW58NeiI_dIY8hA/HdbKGaDNNNwQiJx_ltmRKXP_g7p5b_q53F-q4s0v2V8', 'filename': 'IMG_8199.jpeg', 'size': 54317, 'type': 'image/jpeg', 'thumbnails': {'small': {'url': 'https://v5.airt

In [225]:
_ = response

In [226]:
pprint(response)

Response(response='John Moriarty works on ML algorithms, as indicated by his '
                  'expertise as an "AI / ML specialist researcher".',
         source_nodes=[NodeWithScore(node=TextNode(id_='b4febc4f-cf9c-4cb7-8d43-bd966efecd0c', embedding=None, metadata={'Name': 'Albert Lie', "What's the link to your LinkedIn?": 'https://www.linkedin.com/in/alberttriadrian/', 'What are your areas of expertise you have (select max 4 please)': ['Backend software dev', 'AI Engineer', 'Idea validating']}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='{\n  "Name": "Albert Lie",\n  "What\'s the link to your LinkedIn?": "https://www.linkedin.com/in/alberttriadrian/",\n  "What are your areas of expertise you have (select max 4 please)": [\n    "Backend software dev",\n    "AI Engineer",\n    "Idea validating"\n  ],\n  "What will you build": "AI employees for freight forwarders"\n}', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\

In [6]:
"""Airtable reader."""
from typing import List

from llama_index.core.readers.base import BaseReader
from llama_index.core.schema import Document
import pandas as pd


class CustomAirtableReader(BaseReader):
    """Airtable reader. Reads data from a table in a base.

    Args:
        api_key (str): Airtable API key.
    """

    def __init__(self, api_key: str) -> None:
        """Initialize Airtable reader."""

        self.api_key = api_key

    def load_data(self, base_id: str, table_id: str) -> List[Document]:
        """Load data from a table in a base

        Args:
            table_id (str): Table ID.
            base_id (str): Base ID.
        Returns:
            List[Document]: List of documents.
        """
        from pyairtable import Table

        table = Table(self.api_key, base_id, table_id)
        all_records = table.all()
        return [Document(text=f"{all_records}", extra_info={})]
    
    def get_airtable_df(self, base_id: str, table_id: str) -> pd.DataFrame:
        from pyairtable import Table

        table = Table(self.api_key, base_id, table_id)
        all_records = table.all()
        df = pd.read_json(all_records)

In [ ]:
pprint(response)

In [38]:
# https://docs.llamaindex.ai/en/latest/examples/agent/openai_agent.html#

import json
from typing import Sequence, List

from llama_index.llms.openai import OpenAI
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool, FunctionTool

import nest_asyncio

nest_asyncio.apply()

class YourOpenAIAgent:
    def __init__(
        self,
        tools: Sequence[BaseTool] = [],
        llm: OpenAI = OpenAI(temperature=0, model="gpt-4"),
        chat_history: List[ChatMessage] = [],
    ) -> None:
        self._llm = llm
        self._tools = {tool.metadata.name: tool for tool in tools}
        self._chat_history = chat_history

    def reset(self) -> None:
        self._chat_history = []

    def chat(self, message: str) -> str:
        chat_history = self._chat_history
        chat_history.append(ChatMessage(role="user", content=message))
        tools = [
            tool.metadata.to_openai_tool() for _, tool in self._tools.items()
        ]

        ai_message = self._llm.chat(chat_history, tools=tools).message
        additional_kwargs = ai_message.additional_kwargs
        chat_history.append(ai_message)

        tool_calls = ai_message.additional_kwargs.get("tool_calls", None)
        # parallel function calling is now supported
        if tool_calls is not None:
            for tool_call in tool_calls:
                function_message = self._call_function(tool_call)
                chat_history.append(function_message)
                ai_message = self._llm.chat(chat_history).message
                chat_history.append(ai_message)

        return ai_message.content

    def _call_function(self, tool_call: dict) -> ChatMessage:
        id_ = tool_call["id"]
        function_call = tool_call["function"]
        tool = self._tools[function_call["name"]]
        output = tool(**json.loads(function_call["arguments"]))
        return ChatMessage(
            name=function_call["name"],
            content=str(output),
            role="tool",
            additional_kwargs={
                "tool_call_id": id_,
                "name": function_call["name"],
            },
        )

In [39]:
your_agent = YourOpenAIAgent(tools=[semantic_query_engine_tool, db_query_engine_tool])

In [40]:
your_agent.chat("How many people live in Tokyo?")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


TypeError: 'ChatCompletionMessageToolCall' object is not subscriptable